In [16]:
# importing the pandas module for
# data frame
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score




# load the data set into train variable.
train = pd.read_csv('/kaggle/input/vehicle-count/traffic.csv')
train = train.drop('ID', axis=1)

# display top 5 values of data set
train.head()



,DateTime,Junction,Vehicles
0,2015-11-01 00:00:00,1,15
1,2015-11-01 01:00:00,1,13
2,2015-11-01 02:00:00,1,10
3,2015-11-01 03:00:00,1,7
4,2015-11-01 04:00:00,1,9


In [17]:
# function to get all data from time stamp

# get date
def get_dom(dt):
	return dt.day

# get week day
def get_weekday(dt):
	return dt.weekday()

# get hour
def get_hour(dt):
	return dt.hour

# get year
def get_year(dt):
	return dt.year

# get month
def get_month(dt):
	return dt.month

# get year day
def get_dayofyear(dt):
	return dt.dayofyear

# get year week
def get_weekofyear(dt):
	return dt.weekofyear


train['DateTime'] = train['DateTime'].map(pd.to_datetime)
train['date'] = train['DateTime'].map(get_dom)
train['weekday'] = train['DateTime'].map(get_weekday)
train['hour'] = train['DateTime'].map(get_hour)
train['month'] = train['DateTime'].map(get_month)
train['year'] = train['DateTime'].map(get_year)
train['dayofyear'] = train['DateTime'].map(get_dayofyear)
train['weekofyear'] = train['DateTime'].map(get_weekofyear)

# display
train.head()


,DateTime,Junction,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-11-01 00:00:00,1,15,1,6,0,11,2015,305,44
1,2015-11-01 01:00:00,1,13,1,6,1,11,2015,305,44
2,2015-11-01 02:00:00,1,10,1,6,2,11,2015,305,44
3,2015-11-01 03:00:00,1,7,1,6,3,11,2015,305,44
4,2015-11-01 04:00:00,1,9,1,6,4,11,2015,305,44


In [18]:
train.to_csv('file1.csv')

In [19]:

# display top 5 values of data set
# train = train[train['Junction'] == 1]
train.head()

,DateTime,Junction,Vehicles,date,weekday,hour,month,year,dayofyear,weekofyear
0,2015-11-01 00:00:00,1,15,1,6,0,11,2015,305,44
1,2015-11-01 01:00:00,1,13,1,6,1,11,2015,305,44
2,2015-11-01 02:00:00,1,10,1,6,2,11,2015,305,44
3,2015-11-01 03:00:00,1,7,1,6,3,11,2015,305,44
4,2015-11-01 04:00:00,1,9,1,6,4,11,2015,305,44


In [20]:
# train = train.drop(['Junction'], axis=1)
print(train.columns)

Index(['DateTime', 'Junction', 'Vehicles', 'date', 'weekday', 'hour', 'month',
       'year', 'dayofyear', 'weekofyear'],
      dtype='object')


In [21]:
# there is no use of DateTime module
# so remove it
train = train.drop(['DateTime'], axis=1)

# separating class label for training the data
# train1 = train.drop(['Vehicles'], axis=1)

# class label is stored in target
train = train['Vehicles']


In [22]:
print(train)

0        15
1        13
2        10
3         7
4         9
         ..
48115    11
48116    30
48117    16
48118    22
48119    12
Name: Vehicles, Length: 48120, dtype: int64


In [23]:
print(train.values)

[15 13 10 ... 16 22 12]


In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU
from sklearn.model_selection import train_test_split
# data_normalized = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))
data=train.values
# Define a function to create sequences of past observations
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Past observations
        y.append(data[i+seq_length])    # Next observation (count of vehicles)
    return np.array(X), np.array(y)

# Define the sequence length (number of past observations to consider)
seq_length = 48  # You want to use the past 7 observations

# Create sequences of past observations
X, y = create_sequences(train.values, seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_test,X_val,y_test,y_val=train_test_split(X,y,test_size=0.5,shuffle=False)

import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout, Input, GlobalAveragePooling1D, Concatenate, Flatten

def transformer_encoder(inputs, num_heads, ff_dim, num_layers, dropout_rate):
    x = inputs
    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x, return_attention_scores=False)
        attn_output = Dropout(dropout_rate)(attn_output)
        out1 = LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = Dense(ff_dim, activation='relu')(out1)
        ffn_output = Dense(x.shape[-1])(ffn_output)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        x = LayerNormalization(epsilon=1e-6)(out1 + ffn_output)
    return x

def build_model(input_shape, num_heads, ff_dim, num_layers, dropout_rate):
    inputs = Input(shape=input_shape)
    x = transformer_encoder(inputs, num_heads, ff_dim, num_layers, dropout_rate)
    y=GRU(units=128)(inputs)
    z=LSTM(units=128)(inputs)
    x=Concatenate()([Flatten()(x),Flatten()(y)])
    x=Concatenate()([x,Flatten()(z)])
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1)(x)
    return tf.keras.models.Model(inputs, outputs)

# Example usage:
input_shape = (seq_length, 1)  # Define input shape according to your data
num_heads = 4          # Number of attention heads
ff_dim = 64            # Dimension of the feed-forward network
num_layers = 2         # Number of transformer layers
dropout_rate = 0.1     # Dropout rate

model = build_model(input_shape, num_heads, ff_dim, num_layers, dropout_rate)
model.compile(optimizer='adam', loss='mae')

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model with validation data and callbacks
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Once trained, you can use this model to predict future observations
# For example, if you have new data X_new with shape (n_samples, seq_length, n_features), you can predict as follows:
# y_pred = model.predict(X_new)

Epoch 1/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 28s 18ms/step - loss: 5.6086 - val_loss: 2.6446
Epoch 2/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.8213 - val_loss: 2.5280
Epoch 3/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.7111 - val_loss: 2.5773
Epoch 4/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.6827 - val_loss: 2.4745
Epoch 5/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.6364 - val_loss: 2.4667
Epoch 6/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.6090 - val_loss: 2.4017
Epoch 7/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.6019 - val_loss: 2.4434
Epoch 8/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.5625 - val_loss: 2.4005
Epoch 9/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.5143 - val_loss: 2.4406
Epoch 10/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.5428 - val_loss: 2.4365
Epoch 11/50
1202/1202 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 2.4728 - val_loss: 2.4143
Epoch 12